In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

# === Globale Variablen ===
instance_file_path = None
output = widgets.Output()

# === Auswahlfeld ===
option_selector = widgets.ToggleButtons(
    options=[("Use example file", "example"), ("Upload your own", "upload")],
    description="Select input:",
    style={'description_width': 'initial'}
)

# === Upload-Widget (immer sichtbar) ===
upload_widget = widgets.FileUpload(
    accept='.ttl',
    multiple=False,
    description='Upload TTL file'
)

# === Continue-Button ===
continue_button = widgets.Button(description="Continue", button_style='primary')

# === Auswahlhandler ===
def on_option_change(change):
    output.clear_output()
    if change['new'] == "example":
        global instance_file_path
        instance_file_path = "Instance_Files/swemls-instances.ttl"
        with output:
            print(f"✅ Selected example file: {instance_file_path}")
    elif change['new'] == "upload":
        with output:
            print("📤 Please upload your TTL file using the field below.")

option_selector.observe(on_option_change, names='value')

# === Upload-Handler ===
def on_upload_change(change):
    global instance_file_path
    uploaded = list(upload_widget.value.values())[0]
    instance_file_path = uploaded['metadata']['name']
    with open(instance_file_path, 'wb') as f:
        f.write(uploaded['content'])
    output.clear_output()
    with output:
        print(f"✅ File uploaded and saved as: {instance_file_path}")

upload_widget.observe(on_upload_change, names='value')

# === Continue-Handler ===
def on_continue_clicked(b):
    output.clear_output()
    if not instance_file_path or not os.path.exists(instance_file_path):
        with output:
            print("❌ No valid file selected or uploaded.")
    else:
        with output:
            print(f"📄 Ready to work with: {instance_file_path}")

continue_button.on_click(on_continue_clicked)

# === Anzeige aller Elemente ===
display(option_selector, upload_widget, continue_button, output)


ModuleNotFoundError: No module named 'rdflib'